In [1]:
import glob
import os
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

In [2]:
df = pd.read_csv("data.csv")

In [3]:
df.head

<bound method NDFrame.head of            a1    b1    c1    d1    e1    f1 g1    h1 a2    b2  ... to_h7  \
0           R     N     B     Q     K     B  N     R  P     P  ...     0   
1           R     N     B     Q     K     B  N     R  P     P  ...     0   
2           R     N     B     Q     K     B  N     R  P     P  ...     0   
3           R     N     B     Q     K     B  N     R  P     P  ...     0   
4           R     N     B     Q     K     B  N     R  P     P  ...     0   
...       ...   ...   ...   ...   ...   ... ..   ... ..   ...  ...   ...   
1685519  None  None  None  None  None  None  K  None  P  None  ...     0   
1685520  None  None  None  None  None  None  K  None  P  None  ...     0   
1685521  None  None  None  None  None  None  K  None  P  None  ...     0   
1685522  None  None  None  None  None  None  K  None  P  None  ...     0   
1685523  None  None  None  None  None  None  K  None  P  None  ...     0   

        to_a8 to_b8 to_c8 to_d8 to_e8 to_f8 to_g8 to_h8 g

In [4]:
df = shuffle(df)

In [359]:
df = train[:100]

In [360]:
from sklearn.model_selection import train_test_split
train, test, = train_test_split(df, test_size=0.2,)

In [361]:
features = list(train.iloc[:, 0:192].columns)

In [362]:
X = train[features]
y = train['good_move']

In [363]:
categorical_columns = list(X.iloc[:, 0:63].columns)
numerical_columns = list(X.iloc[:, 64:192].columns)
feature_columns = []

for feature_name in categorical_columns:
    vocabulary = X[feature_name].unique()
    feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))


for feature_name in numerical_columns:
    feature_columns.append(tf.feature_column.numeric_column(feature_name,dtype = tf.float32))

In [364]:
def make_input_fn(data_df, label_df, num_epochs = 10, shuffle = True, batch_size = 32):
    def input_function():
        ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
        if shuffle:
            ds = ds.shuffle(1000)
        ds = ds.batch(batch_size).repeat(num_epochs)
        return ds
    return input_function

In [365]:
def split_into_batches(df, batch_size=200000):
    nb_rows = len(df.index)
    intervals = []
    
    for i in range(0, nb_rows + 1, batch_size):
        intervals.append(i)
  
    if(intervals[-1] != nb_rows):
        intervals.append(nb_rows)
  
    batches_X = []
    batches_y = []
  
    for i in range(0, len(intervals) - 1):
        batches_X.append(df.iloc[intervals[i]:intervals[i + 1], :][features])
        batches_y.append(df.iloc[intervals[i]:intervals[i + 1], :]['good_move'])

    return batches_X, batches_y

In [366]:
batches_X, batches_y = split_into_batches(train)

In [367]:
xval, yval = split_into_batches(test)

In [368]:
len(train)

80

In [369]:
##initialize a classifier
linear_est = tf.estimator.LinearClassifier(feature_columns = feature_columns, model_dir='model20')


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'model20', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [370]:
input_functions = []
for df_X, df_y in zip(batches_X, batches_y):
    print(df_X)
    print(type(df_X))
    input_functions.append(make_input_fn(df_X, df_y))

           a1    b1    c1    d1    e1    f1    g1    h1    a2    b2  ...  \
153873   None     B  None  None     R  None     K  None  None     P  ...   
582826   None  None  None  None  None  None  None  None     q  None  ...   
716292   None  None  None  None  None     K  None  None     P  None  ...   
1423508     R     N     B     Q     K     B  None     R     P     P  ...   
829625      R  None     B  None  None  None  None     K     P     P  ...   
...       ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
929511      R  None  None  None     R     B     K  None  None  None  ...   
203203      R     N     B     Q     K     B     N     R     P     P  ...   
718611      R     N     B     Q     K     B  None     R     P     P  ...   
910872   None  None     K     R  None  None     R  None  None     P  ...   
1657174     R     N     B     Q     K     B  None     R     P     P  ...   

        to_g7 to_h7 to_a8 to_b8 to_c8 to_d8 to_e8 to_f8 to_g8 to_h8  
153873      0    

In [371]:
type(xval[0])

pandas.core.frame.DataFrame

In [372]:
type(batches_X)

list

In [373]:
# train the model on all the input functions
i = 1
for input_function in input_functions:
    print('<======================================== NEW BATCH ========================================>')
    print('Batch: ' + str(i))
    i = i + 1
    linear_est.train(input_function)
  
 # save the model
serving_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
  tf.feature_column.make_parse_example_spec(feature_columns))



<======================================== NEW BATCH ========================================>
Batch: 1
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into model20/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 0.6931472, step = 0
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 30...
INFO:tensorflow:Saving checkpoints for 30 into model20/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 30...
INFO:tensorflow:Loss for final step: 0.08498185.


In [374]:
estimator_base_path = 'model20'
estimator_path = linear_est.export_saved_model(estimator_base_path, serving_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: ['serving_default', 'classification']
INFO:tensorflow:Signatures INCLUDED in export for Regress: ['regression']
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from model20/model.ckpt-30
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: model20/temp-1619974610/saved_model.pb


In [375]:
## validation

In [376]:
eval_input_fn = make_input_fn(xval[0], yval[0])

In [377]:
result = linear_est.evaluate(eval_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-05-03T00:57:05Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model20/model.ckpt-30
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 3.76227s
INFO:tensorflow:Finished evaluation at 2021-05-03-00:57:08
INFO:tensorflow:Saving dict for global step 30: accuracy = 1.0, accuracy_baseline = 1.0, auc = 0.0, auc_precision_recall = 0.0, average_loss = 0.01188066, global_step = 30, label/mean = 0.0, loss = 0.01188066, precision = 0.0, prediction/mean = 0.0117549375, recall = 0.0
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 30: model20/model.ckpt-30
